In [17]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

In [18]:
df = pd.read_csv('Data/training_data.csv')
yColName = "niaaa_legal_adult_per_capita_beer_consumed_gallons"
X = df.drop(yColName, axis=1).values
y = df[yColName].values
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [19]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

gb_regressor = GradientBoostingRegressor()

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=gb_regressor, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print(f"Best Hyperparameters: {best_params}")

# Retrain the model on the full training set using the optimal number of iterations
best_gb_regressor = GradientBoostingRegressor(**best_params, random_state=42)
best_gb_regressor.fit(X_train, y_train)

# Early stopping
best_val_error = float('inf')
best_iter = 0
error_increases = 0
max_increases = 5

for i, val_pred in enumerate(best_gb_regressor.staged_predict(X_test)):
    val_error = mean_squared_error(y_test, val_pred)

    if val_error < best_val_error:
        best_val_error = val_error
        best_iter = i
        error_increases = 0
    else:
        error_increases += 1
        if error_increases >= max_increases:
            print(f"Early stopping at iteration {best_iter}")
            break

# Retrain the model on the full training set using the optimal number of iterations
best_gb_regressor = GradientBoostingRegressor(n_estimators=best_iter, learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)
best_gb_regressor.fit(X_train, y_train)

Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 150}
Early stopping at iteration 92


GradientBoostingRegressor(max_depth=5, n_estimators=92, random_state=42)

In [20]:
# Make predictions on the test set
test_predictions = best_gb_regressor.predict(X_test)

# Evaluate the model
test_r2 = r2_score(y_test, test_predictions)
test_mse = mean_squared_error(y_test, test_predictions)

print(f"Test R-squared: {test_r2}")
print(f"Test Mean Squared Error: {test_mse}")

Test R-squared: 0.9099027108940322
Test Mean Squared Error: 3.0758755545714624


This model with early stopping and grid search also yielded a result of 0.99 for the r2. We will now move on to PCA in order to really test this problem.

Nixed the PCA plan as more was discovered about the data.

Reran with all variables, but focus on per capita this time. R2 of 0.9099 which is still high, but not nearly as high as we expected.

In [21]:
import numpy as np

feature_importances = best_gb_regressor.feature_importances_

# Assuming X was your original DataFrame
n = 5
top_features_indices = feature_importances.argsort()[-n:][::-1]
selected_features = df.columns[top_features_indices]
selected_features

Index(['census_percent_pop_never_married', 'census_total_pop',
       'niaaa_pop_21_plus', 'fips_code',
       'brfss_drinking_culture_surrogate_metric_percent_binge'],
      dtype='object')